# 전체 크롤링 코드

In [8]:
import os
import csv
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "C:/chromedriver"
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

reviews = []  # 중복 리뷰를 저장하지 않기 위한 리스트 생성

def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    place_infos = [input("")]

    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)
        
    # 저장된 리뷰를 CSV 파일에 저장
    save_to_csv(place, reviews)
        
    driver.quit()
    print("finish")

def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists[:5])

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()

def save_to_csv(place, reviews):
    file_name = f"{place}_reviews.csv"
    with open(file_name, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['review']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for review in reviews:
            writer.writerow({'review': review})
            print(f"{file_name} saved.")


def crawling(place, place_lists):

    while_flag = False
    for i, place in enumerate(place_lists):
        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print(i+1,'.', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 리뷰 페이지 찾기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('후기 더보기').send_keys(Keys.ENTER) # 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        crawl_next_reviews(place_name)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환

def extract_review(place_name):
    global driver, reviews # reviews 리스트를 사용하기 위해 global로 선언
    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 현재 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span')  # 리뷰
            if comment:
                comment_text = comment[0].text.strip()  # 공백 제거
                if comment_text and comment_text not in reviews:  # comment_text가 비어있지 않고 중복이 아닌 경우에만 출력
                    reviews.append(comment_text)  # 중복이 아닌 리뷰를 reviews 리스트에 추가
                    print(comment_text)
    else:
        print('no review in extract')
        ret = False

    return ret

def crawl_next_reviews(place_name):
    global driver
    max_clicks = 5  # 최대 클릭 횟수 설정
    click_count = 0 # 현재 클릭 횟수 초기화
    while click_count < max_clicks:
        try:
            driver.find_element_by_link_text('후기 더보기').send_keys(Keys.ENTER)  # 후기 더보기 버튼 클릭
            sleep(1)
            if not extract_review(place_name):  # 리뷰 추출
                break
            click_count += 1  # 클릭 횟수 증가

        except (NoSuchElementException, ElementNotInteractableException):
            print("no more reviews to load")
            break

            
            
if __name__ == "__main__":
    main()
    

 제주도 해수욕장


##### 0
1 . 이호테우해수욕장
말 필요 없음
이호테우 해변 비오는 날 방문했는데 운치있고 넘 좋네요!!!!
좋았어요 ㅎㅎ 말등대도보공
귀여운 말등대가 있는 해수욕장🪅아이들 놀기 좋은 해수풀장이 있어요~
제주도 마지막 여행지
그냥 시원한 맛에 갈만함 사람 꽤 있는 편이고 말 등대까지 산책하기 괜찮음 -겨울방문- 여름에는 해수욕 하기 괜찮을듯 잔잔하니 아이들이 놀기 괜찮음
https://youtu.be/R4hXLXd9s9Aok차박 이호태우해수욕장 유튜브 검색하시면 동영상 올렸습니다 낮과밤 차박지
꼭 건강 해져서 다시한번 가고싶다 병원 밖으로 나가서  제주도로 가자
서핑스팟으로서0점. 라인업에 사람 30명중문처럼 꼽주는 쓰레기 다수물론 좋은사람이 많지만까맣고 얼굴 찌뿌리고 있는 사람 많은 해수욕장에는서핑하러 가는거 아님
물이 맑아서 속이 다 보이는 곳!말 동상으로 유명해서 와보고 싶었던 곳!!플라스틱에 대한 경각심을 불러일으키는 곳!
인생샷 찍으러왔니? 응 여기아니야..처음가봤는데 말등대가 왜이리 멀어.. 이호항 으로 가야댐 가즈아!
제주도 바다 중 물색은 가장 안 좋은듯...
알바생들이 손님 뒷담까면서 욕하네요^^ 담부터는 안갈듯요~ 흔한 일진알바생인지  불쾌하네용^^
좋았음
제주바다중에 가장더러운 해수욕장?  절대 가지마시길 가실거면  협재  김녕 함덕 표선이  제일  좋고 깨끗함
좀 걷긴 해야하지만 버스 정류장이 근처에 있어서 접근성이 좋았어요 하지만 물이 좀 더럽고(해조류 많음!) 화장실에 냄새가 났어요 ㅜㅜ 그래두 가볍게 물놀이 하기엔 좋아용!!
예쁜 뷰가 보이는 항구...
물더러움 냄새남애기들은 이호테우 해수풀장에서 노세용
관리가 제대로 안되고 있는 거 같아요;; 서핑하러 갔다가 물이 더러워서 한시간 만에 나왔어요물에 비닐봉지도 너무 많이 떠다녀서 충격먹고 나왔습니다....
애랑 가서 놀기좋아요 물이 깨끗하진 않고 바닥에 바위 뾰족한게 있어서 물놀이시 조심해야 합니다 공항근처라 바다에서 지나가는 비행기 구경도 재밌어요
바람과 함께하는 해수욕
이호테

In [9]:
import os
from time import sleep
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "C:/chromedriver"
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)

def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')
    search_area.send_keys(place)
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)
    sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem')

    crawling(place, place_lists[:5])
    search_area.clear()

    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        for i in range(2, 6):
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem')

            crawling(place, place_lists[:5])

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()

def crawling(place, place_lists):
    global driver, all_data

    for i, place in enumerate(place_lists):
        place_name = place.select('.head_item > .tit_name > .link_name')[0].text

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])
        sleep(1)

        print(i+1, '.', place_name)        
        
        data_dict = extract_data()

        if data_dict:
            all_data.append(data_dict)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

def find_element(xpath):
    global driver

    try:
        element = driver.find_element_by_xpath(xpath)
        return element.text
    except NoSuchElementException:
        return ""

def extract_data():
    global driver

    place_name = find_element('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2')
    place_location = find_element('//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[1]')
    place_feature = find_element('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/span[1]')
    place_star =  find_element('//*[@id="mArticle"]/div[5]/div[1]/div/em')
    operating_time = find_element('//*[@id="mArticle"]/div[1]/div[2]/div[2]/div/div[1]/ul/li/span/span[1]')

    crawling_data = [place_name, place_location, place_feature, place_star, operating_time]
    
    if place_name:
        data_dict = {"place_name": place_name,
            "place_location": place_location,
            "place_feature": place_feature,
            "place_star": place_star,
            "operating_time": operating_time}
        return data_dict, print(crawling_data)
    else:
        return None

def main():
    global driver, all_data
    all_data = []
    driver.implicitly_wait(1)
    driver.get('https://map.kakao.com/')
    
    place_infos = [input("")]
    place_infos_str = "".join(place_infos)

    for i, place in enumerate(place_infos):
        if i % 4 == 0 and i != 0:
            sleep(1)
        search(place)

    driver.quit()
    print("finish")

    # Save all_data to a CSV file
    if all_data:
        data_frame = pd.DataFrame(all_data)
        data_frame.to_csv(f"{place_infos_str}_reviews.csv", index=False, encoding='utf-8-sig')
        print("Data saved to top_25_places_data.csv")
    else:
        print("No data to save")

if __name__ == "__main__":
    main()